In [ ]:
from YELP_API_KEY import YELP_API_KEY
import csv
import requests
import json
from typing import List, Dict
from bs4 import BeautifulSoup as bs
import re

In [ ]:
def search_nested(nested_dictionary, keys: set, aliases=None, result=[]):
    for key, value in nested_dictionary.items():
        if key in keys and aliases is not None and key in aliases:
            result.append({aliases[key]: value})
        elif key in keys:
            result.append({key: value})
        elif type(value) is dict:
            search_nested(value, keys, aliases, result)
    return result

In [ ]:
def get_business_info(ids: set,
                      keys: set,
                      api_key: str,
                      aliases=None):
    """Makes a request to the Yelp business api and gets the desired params.
    Aliases allows renaming of yelp keys."""
    businesses: List[Dict] = []
    url = 'https://api.yelp.com/v3/businesses/'
    headers = {'Authorization': f'Bearer {YELP_API_KEY}'}
    for id_ in ids:
        response = requests.get(f'{url}{id_}', headers=headers)
        business = {'id': id_}
        for key, value in response.json().items():
            if key in keys and aliases is not None and key in aliases:
                business[aliases[key]] = response.json()[key]
            elif key in keys:
                business[key] = response.json()[key]
            elif type(value) is dict:
                nested_keys = search_nested(value, keys, aliases)
                for result in nested_keys:
                    business.update(result)
        businesses.append(business)
    return businesses

In [ ]:
def scrape_business_info(yelp_url):
    response = requests.get(yelp_url)
    soup = bs(response.text)
    url = soup.find('a', href=re.compile("^\/biz_redir\?url="), role='link')
    if url is not None:
        return url.text
    return url

In [ ]:
csv_file = 'businesses_v2.csv'
ids: set = set()
with open(csv_file, 'r') as csv_file:
    for row in csv.reader(csv_file):
        ids.add(row[0])

In [ ]:
keys = {
    'name',
    'image_url',
    'url',
    'phone',
    'categories',
    'address1',
    'city',
    'zip_code',
    'country',
    'state',
    'latitude',
    'longitude',
    'photos',
    'price',
}
aliases = {
    'url': 'yelp_url',
    'address1': 'street_address',
    'zip_code': 'postal_code',
    'state': 'province',
}
businesses = get_business_info(ids, keys, YELP_API_KEY, aliases)
for business in businesses:
    business['url'] = scrape_business_info(business['yelp_url'])

In [ ]:
for business in businesses:
    business['url'] = scrape_business_info(business['yelp_url'])